# Census Data
Collects ACS data from census.gov API and merges it into one table.

If you want to collect your own census data, you'll need to regiester for an [API key](https://api.census.gov/data/key_signup.html), and assign it as the environment variable `CENSUS_API_KEY`. 

This is not necessary, as all outputs are calcualted and saved in this repository already.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import requests
import glob as glob
import gzip
import multiprocess
from multiprocess import Pool

from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd

## Download ACS data

In [3]:
# ouputs
data_dir = '../data/input/census/acs5'
os.makedirs(data_dir, exist_ok=True)

# inputs
fn_income = '../data/census/state2income.json'
input_files = glob.glob('../data/intermediary/isp/*/*/*.geojson.gz')
fips = set([_.split('/')[-1].split('.geojson')[0][:5] for _ in input_files])
fips = set([_ for _ in fips if _ != 'spotc'])
state2income = json.load(open(fn_income, 'r'))

# params
recalculate = False
n_jobs = 8
API_KEY = os.environ.get('CENSUS_API_KEY')
# assert API_KEY

### Making API calls

In [12]:
# outputs
fn_out_acs = '../data/intermediary/census/aggregated_tables.csv.gz'
os.makedirs(os.path.dirname(fn_out_acs), exist_ok=True)

In [5]:
# the three ACS tables and columns we're going to get.
acs_tables = [
    {
        "display_name": "race_ethnicity",
        "table_name": "B03002",
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B03002.html",
        "columns": {
            "block group": "block_group",
            "B03002_001E": "race_total_estimate",
            "B03002_003E": "race_white_alone",
            "B03002_004E": "race_black_alone",
            "B03002_005E": "race_aindian_alone",
            "B03002_006E": "race_asian_alone",
            "B03002_007E": "race_pacific_islander_native_hawaiian_alone",
            "B03002_008E": "race_some_other_alone",
            "B03002_009E": "race_two_or_more_alone",
            "B03002_012E": "race_latino_alone",
        }
    },
    {
        "display_name": "household_income_2",
        "table_name": 'B19013',
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B19013.html",
        "columns": {
            "block group": "block_group",
            "B19013_001E": "median_household_income",
        }
    },
    {
        "display_name": "internet_subscription",
        "table_name": 'B28002',
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B28002.html",
        "columns": {
            "block group": "block_group",
            "B28002_001E": "internet_total_estimate",
            "B28002_002E": "internet_subscriptions_any",
            "B28002_004E": "internet_broadband",
            "B28002_005E": "internet_mobile",
            "B28002_006E": "internet_mobile_only",
            "B28002_013E": "internet_no_access",
        }
    }
]

In [6]:
def download_acs(column: str= "B03002_001E",
                 geography: str = "block group",
                 year: int= 2019, 
                 state: int= 55,
                 county: [int, str]= 1,
                 data_dir: str= 'data/',
                 api_key: str= None,
                 debug: bool= False) -> dict:
    """
    geography can also be "tract",
    column is the ACS "table name"
    """
    import os
    import pandas as pd
    import requests
    
    def make_acs_request(year, column, geography, state, county, api_key, debug=False):
        """
        Formats the API call and make the reuqest
        """
        county = str(county).zfill(3)
        url = (f"https://api.census.gov/data/{year}/acs/acs5?get={column}&"
               f"for={geography}:*&in=state:{state}%20county:{county}&key={api_key}")
        if debug:
            print(url)
        return requests.get(url)

    
    # check if the file exists...
    table = column.split('_')[0]
    geography_ = geography.replace(' ', '_')
    fn_out = f"{data_dir}/{geography_}/{year}/{state}/{county}/{table}/{column}.csv.gz"
    if os.path.exists(fn_out):
        return 1
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    print(f"collecting {fn_out}")
    # make the request
    resp = make_acs_request(year, column, geography, state, county, api_key, debug)
    
    # validate the response and save it
    if resp.status_code != 200:
        pd.DataFrame([]).to_csv(fn_out, index=False, compression='gzip')
        return 0
    _data = resp.json()
    _df = pd.DataFrame(_data[1:], columns=_data[0])
    _df.to_csv(fn_out, index=False, compression='gzip')
    return 1

In [7]:
def make_acs_request(year, column, geography, state, county, api_key, debug=False):
        """
        Formats the API call and make the reuqest
        """
        county = str(county).zfill(3)
        url = (f"https://api.census.gov/data/{year}/acs/acs5?get={column}&"
               f"for={geography}:*&in=state:{state}%20county:{county}&key={api_key}")
        if debug:
            print(url)
        return requests.get(url)


To speed things up, we're going to use multiprocessing to collect these files.
First, we must create a list of arguments for each API request.

In [8]:
year = 2019
debug = False
args = []
for fip in fips:
    state = fip[:2]
    county = fip[2:]
    for geography in ['block group']:
        for table in acs_tables:
            table_name = table["display_name"]
            columns = [c for c in table["columns"].keys() if c != "block group"]
            for column in columns:
                args.append([column, geography, year, state, county, data_dir, API_KEY, debug])

f"We must make {len(args)} API calls."

'We must make 16 API calls.'

In [13]:
# use multiprocessing to make the requests...
def collect_data():
    with multiprocess.get_context("spawn").Pool(n_jobs) as pool:
        pool.starmap(download_acs, tqdm(args, total=len(args)))
        
# only make API requests if the output file doesn't exists or if we want to recalculate.
if not os.path.exists(fn_out_acs) or recalculate:
    collect_data()

## Data processing
With the ACS data collected, we're now going to calculate the racial demographics, income levels, and broadband pentration for each block group.

In [14]:
def get_relevant_block_groups(verbose=False):
    """Get the block groups in the data we have"""
    files_input = glob.glob("../data/input/isp/*/*/*.geojson.gz")    
    block_groups = set([f.split('/')[-1].split('.geojson')[0] for f in files_input])
    block_groups = set([bg for bg in block_groups if len(bg) == 12])
    if verbose:
        print(f"Found {len(block_groups)} block groups.")
    
    return block_groups

In [15]:
def get_geoid(row):
    """
    Explictly cast block group-level geoid
    """
    state = int(row['state'])
    county = int(row['county'])
    tract = int(row['tract'])
    block = int(row['block_group'])
    
    return f"{state:02}{county:03}{tract:06}{block:01}"

def percent_non_white(row):
    """
    Percentage of area that are not non-Hispanic white
    """
    total = row['race_total_estimate']
    white = row['race_white_alone']
    try:
        perc_non_white =  (total - white) / total
        return max(0, min(perc_non_white, 1))
    except ZeroDivisionError:
        return None
    
def income_level(row):
    """
    Calculate median household income compared to city median income.
    We set None for skip values set by the census bureau.
    """
    state = row['state']
    if row['median_household_income'] == -666666666:
        return None
    median_income = state2income.get(str(state), {})
    if median_income:
        return row['median_household_income'] / median_income
    return None

def dollars_diff_median(row):
    """
    Get dollars below median city income.
    """
    state = row['state']
    if row['median_household_income'] == -666666666:
        return None
    median_income = state2income.get(str(state), {})
    if median_income:
        return median_income - row['median_household_income']
    return None

In [16]:
if not os.path.exists(fn_out_acs) or recalculate:
    # This merges and concats each ACS table we collected.
    geography = "block group"
    df = pd.DataFrame([])
    for table in acs_tables:
        col2rename = table['columns']
        table_name = table['table_name']
        cols = [c for c in col2rename.keys() if c != "block group"]
        for column in tqdm(cols):
            files = glob.glob(f"../data/input/census/acs5/{geography.replace(' ', '_')}/{year}/*/*/{table_name}/{column}.csv.gz")
            data = []
            for fn in files:
                tp = pd.read_csv(fn, compression='gzip')
                if tp.empty:
                    print(fn)
                data.extend(tp.to_dict("records"))
            tmp = pd.DataFrame(data)
            tmp.columns = [col2rename.get(c, c) for c in tmp.columns]
            if df.empty:
                df = tmp
            else:
                df = df.merge(tmp, on= ["state", "county", "tract", "block_group"], how='outer')

    # process the data
    df['race_perc_non_white'] = df.apply(percent_non_white, axis=1)
    df['internet_perc_broadband'] = df['internet_broadband'] / df['internet_total_estimate']
    df['geoid'] = df.apply(get_geoid, axis=1)
    
    # For whatever reason, calling these functions can return None.
    # check how many null values they produce, and run again if there are a lot of nulls.
    df['income_dollars_below_median'] = df.apply(dollars_diff_median, axis=1)
    df['income_lmi'] = df.apply(income_level, axis=1)
    
    df.to_csv(fn_out_acs, index=False, compression='gzip')
else:
    df = pd.read_csv(fn_out_acs, compression='gzip')

## Processing raw shape file for maps

[TIGER](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html) shape files were downloaded and stored in one place (`../data/input/census/shape/raw`).

In [18]:
# ouputs
fn_out_shp = '../data/intermediary/census/2019_acs_5_shapes.geojson.gz'
pattern_geojson = '../data/input/census/shape/preprocessed/*.geojson'

# inputs
files_shape = glob.glob('../data/input/census/shape/raw/tl_2019_*_bg.zip')
len(files_shape)

46

We're going to filter out block groups that aren't in our investigation, merge ACS demographic data, and combine the shapes into one file.

the zip files need to be converted into geoJSON (via geopandas `gpd`) to map-able.

In [20]:
# convert zip files to geoJSON
for fn in tqdm(files_shape):
    fn = os.path.abspath(fn)
    fn_out = fn.replace('/raw/', '/preprocessed/').replace('.zip', '.geojson')
    if os.path.exists(fn_out):
        continue
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    # shp = gpd.read_file('zip:///' + fn)
    shp = gpd.read_file(fn)
    shp.to_file(fn_out, driver='GeoJSON')

  0%|          | 0/46 [00:00<?, ?it/s]

  7%|▋         | 3/46 [02:39<38:07, 53.20s/it]


KeyboardInterrupt: 

In [ ]:
if not os.path.exists(fn_out_shp) or recalculate:
    df = pd.read_csv(fn_out_acs, compression='gzip')
    df.geoid = df.geoid.apply(lambda x: f"{x:012d}")

    block_groups = get_relevant_block_groups(verbose=True)    
    geoid2meta = {row['geoid']: {
        'race_perc_non_white' : row['race_perc_non_white'], 
        'income_dollars_below_median' : row['income_dollars_below_median'],
        'median_household_income' : row['median_household_income']
    }  for i, row in df[df.geoid.isin(block_groups)].iterrows()}
    len(geoid2meta)
    
    save_all = []
    files_geojson = glob.glob(pattern_geojson)
    geography = "block group"
    for fn_geojson in tqdm(files_geojson):
        fn_out =  f'../data/intermediary/maps/acs/{os.path.basename(fn_geojson)}'
        if not os.path.exists(fn_out) or recalculate:
            os.makedirs(os.path.dirname(fn_out), exist_ok=True)
            to_save = []
            with open(fn_geojson , 'r') as f:
                geojson = json.load(f)
                for i, feature in enumerate(geojson['features']):
                    featureProperties = feature['properties']
                    if geography == "block group":
                        geoid = featureProperties['GEOID']
                    featureData = geoid2meta.get(geoid, {})
                    for key in featureData.keys():
                        featureProperties[key] = featureData[key]
                    if featureData:
                        to_save.append(feature)
                        save_all.append(feature)

                # save geoJSON with acs data merged in
                geojson['features'] = to_save
                with open(fn_out, 'w') as f:
                    f.write(json.dumps(geojson).replace('NaN', '""'))
    geojson['features'] = save_all
    with gzip.open(fn_out_shp, 'wt') as f:
        f.write(json.dumps(geojson).replace('NaN', '""'))

## population density & number of competing ISPs
Final tweaks to the ACS data to be used for analysis.

We reference TIGER shape files to calculate population density using the the area of land for each census block group and the estimated population. We also merge the number of competing ISPs for each block group according to the FCC's Form 477.

In [ ]:
# ouputs
fn_out_features = '../data/intermediary/census/aggregated_tables_plus_features.csv.gz'
fn_providers = '../data/intermediary/fcc/bg_providers.csv'

In [ ]:
# inputs
fn_form_477 = '../data/input/fcc/fbd_us_with_satellite_dec2020_v1.csv.gz'

In [ ]:
# get number of competitors from FCC Form 477.
if not os.path.exists(fn_providers) or recalculate:
    # Filter for block groups in our invetigation.
    block_groups = get_relevant_block_groups(verbose=True)
    data = []
    for _df in pd.read_csv(fn_form_477, 
                           compression= 'gzip',
                           encoding= 'unicode_escape', 
                           chunksize= 50000, 
                           dtype= {'BlockCode' : str}):
        _df['block_group'] = _df['BlockCode'].apply(lambda x: x[:12])
        data.extend(_df[_df['block_group'].isin(block_groups)]
                                          .to_dict(orient='records'))
    df_bg = pd.DataFrame(data)
    
    (df_bg[
        (df_bg.Consumer == 1) &
        (~df_bg.TechCode.isin([60,70]))   
    ].groupby('block_group')['ProviderName']
     .nunique().to_frame('n_providers')
     .to_csv(fn_providers)
    )

In [ ]:
def calculate_persons_per_sq_mile(row):
    """
    See formula referenced in these two sources:
    https://acsdatacommunity.prb.org/discussion-forum/f/forum/585/moe-for-population-density
    https://www.census.gov/quickfacts/fact/note/US/LND110210
    1,000,000 * POP / ALAND
    """
    if row['ALAND'] == 0:
        return None
    sq_miles = row['ALAND'] / 1000000
    persons = row['race_total_estimate']
    return persons / sq_miles

In [ ]:
if not os.path.exists(fn_out_features) or recalculate:
    df = pd.read_csv(fn_out_acs, compression='gzip', dtype={'geoid': str})
    df_shp = pd.DataFrame([row['properties'] for row in 
                           json.load(gzip.open(fn_out_shp, 'r'))['features']])
    df_providers = pd.read_csv(fn_providers, dtype={'block_group': str})
    
    df = df.merge(df_shp[['GEOID', 'ALAND']], 
                  how='left', left_on='geoid', right_on='GEOID', 
                  suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
    df['ppl_per_sq_mile'] = df.apply(calculate_persons_per_sq_mile, axis=1)

    df = df.merge(df_providers, 
                  how='left', left_on='GEOID', right_on='block_group', 
                  suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
    
    df.to_csv(fn_out_features, index=False, compression='gzip')